#  Scraping sur Flipkart

Ce notebook a pour objectif de scraper des informations sur Flipkart pour différentes catégories de produits (graphics_cards, laptops, monitors, smart_watches).

**Fonctionnalités :**
- Récupération des détails d'un produit (spécifications, ratings, reviews, etc.).
- Extraction des informations depuis une page de résultats.
- Sauvegarde des données extraites dans des fichiers CSV organisés par catégorie.

**Bibliothèques utilisées :**
- `os`, `time`, `random` pour la gestion du système et des délais.
- `requests` pour envoyer des requêtes HTTP.
- `BeautifulSoup` (via `bs4`) pour parser le HTML.
- `pandas` pour la manipulation et la sauvegarde des données.
- `datetime` pour la gestion des dates.
- `re` pour les expressions régulières.


In [ ]:
import os
import time
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re


## Constants et Headers

Nous définissons ici quelques constantes essentielles :
- **USER_AGENTS** : Une liste de User-Agent pour simuler des requêtes provenant de différents navigateurs.
- **DEFAULT_HEADERS** : Les en-têtes HTTP utilisés pour nos requêtes, incluant un User-Agent choisi aléatoirement.


In [ ]:
# Liste de User-Agent pour varier les requêtes
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1',
]

DEFAULT_HEADERS = {
    'User-Agent': random.choice(USER_AGENTS),
    'Accept-Language': 'en-US, en;q=0.5'
}


## Fonctions Utilitaires

Ces fonctions facilitent l'extraction et le traitement des données :
- **get_text_or_default** : Extrait le texte d'un élément BeautifulSoup ou renvoie une valeur par défaut.
- **wait_random** : Introduit une attente aléatoire pour éviter d'être détecté.
- **extract_specifications** : Extrait les spécifications d'un produit à partir du HTML.


In [ ]:
def get_text_or_default(element, default="Data not available"):
    """Extrait le texte d'un élément BeautifulSoup ou renvoie une valeur par défaut."""
    return element.text.strip() if element else default

def wait_random(min_time=3, max_time=7):
    """Attend un nombre de secondes aléatoire pour éviter la détection."""
    delay = random.randint(min_time, max_time)
    print(f"Waiting {delay} seconds...")
    time.sleep(delay)

def extract_specifications(soup):
    """Extrait les spécifications de la section de détails d'un produit."""
    specifications = {}
    spec_sections = soup.find_all('div', class_='GNDEQ-')

    for section in spec_sections:
        section_title = section.find('div', class_='_4BJ2V+')
        if not section_title:
            continue
        section_title = section_title.text.strip()

        spec_rows = section.find_all('tr', class_='WJdYP6 row')
        for row in spec_rows:
            key_element = row.find('td', class_='+fFi1w col col-3-12')
            value_element = row.find('td', class_='Izz52n col col-9-12')

            if key_element and value_element:
                key = key_element.text.strip()
                value = value_element.text.strip()
                specifications[key] = value

    return specifications


## Scraping des Détails d'un Produit

La fonction `scrape_flipkart_product` effectue les opérations suivantes :
- Envoie une requête HTTP pour obtenir la page du produit.
- Parse le HTML et extrait les spécifications via `extract_specifications`.
- Récupère la note (rating) et le nombre d'avis (reviews) à l'aide d'expressions régulières.
- Retourne un dictionnaire regroupant ces informations.


In [ ]:
def scrape_flipkart_product(product_url):
    """Scrape les informations détaillées d'un produit (spécifications, rating, reviews)."""
    headers = DEFAULT_HEADERS
    try:
        response = requests.get(product_url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        specifications = extract_specifications(soup)

        rating_element = soup.find('div', class_='_3LWZlK')
        rating = get_text_or_default(rating_element)

        reviews_element = soup.find('span', class_='_2_R_DZ')
        reviews_text = get_text_or_default(reviews_element)

        reviews_match = re.search(r'\d+', reviews_text.replace(',', ''))
        reviews = reviews_match.group() if reviews_match else "Data not available"

        return {
            "rating": rating,
            "reviews": reviews,
            **specifications,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error occurred while scraping product {product_url}: {e}")
        return {"rating": "Data not available", "reviews": "Data not available"}


## Scraping d'une Page de Résultats

La fonction `scrape_flipkart_page` :
- Envoie une requête pour obtenir une page de résultats d'une catégorie donnée.
- Identifie les blocs produits et extrait pour chacun les informations telles que le titre, le prix, le rating, les reviews, l'image et l'URL du produit.
- Pour chaque produit, elle peut également appeler `scrape_flipkart_product` afin d'obtenir des spécifications détaillées.


In [ ]:
def scrape_flipkart_page(url, category_name):
    headers = DEFAULT_HEADERS
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        product_blocks = soup.find_all('div', class_='cPHDOP col-12-12')
        if not product_blocks:
            print("No product blocks found on this page.")
            return []

        scraped_items = []
        collection_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        for product in product_blocks:
            if category_name == "graphics_cards":
                title_element = product.find('a', class_='wjcEIp')
                price_element = product.find('div', class_='Nx9bqj')
                rating_element = product.find('div', class_='XQDdHH')
                reviews_element = product.find('span', class_='Wphh3N')
                image_element = product.find('img', class_='DByuf4')
                link_element = product.find('a', class_='VJA3rP')
            elif category_name == "laptops":
                title_element = product.find('div', class_='KzDlHZ')
                price_element = product.find('div', class_='Nx9bqj _4b5DiR')
                rating_element = product.find('div', class_='XQDdHH')
                reviews_element = product.find('span', class_='Wphh3N')
                image_element = product.find('img', class_='DByuf4')
                link_element = product.find('a', class_='CGtC98')
            elif category_name == "monitors":
                title_element = product.find('div', class_='KzDlHZ')
                price_element = product.find('div', class_='Nx9bqj _4b5DiR')
                rating_element = product.find('div', class_='XQDdHH')
                reviews_element = product.find('span', class_='Wphh3N')
                image_element = product.find('img', class_='DByuf4')
                link_element = product.find('a', class_='CGtC98')
            elif category_name == "smart_watches":
                title_element = product.find('a', class_='WKTcLC')
                price_element = product.find('div', class_='Nx9bqj')
                rating_element = product.find('div', class_='XQDdHH')
                reviews_element = product.find('span', class_='Wphh3N')
                image_element = product.find('img', class_='_53J4C-')
                link_element = product.find('a', class_='rPDeLR')

            title = get_text_or_default(title_element)
            price = get_text_or_default(price_element)
            rating = get_text_or_default(rating_element)
            reviews = get_text_or_default(reviews_element)
            image_url = image_element['src'] if image_element else "Image not available"
            product_url = f"https://www.flipkart.com{link_element['href']}" if link_element else "URL not available"

            specifications = scrape_flipkart_product(product_url) if product_url != "URL not available" else {}

            scraped_items.append({
                "title": title,
                "price": price,
                "rating": rating,
                "reviews": reviews,
                "image_url": image_url,
                "product_url": product_url,
                "collection_date": collection_date,
                **specifications,
            })

        return scraped_items

    except requests.exceptions.RequestException as e:
        print(f"Error occurred while scraping {url}: {e}")
        return []


## Gestion de la Numérotation des Scrapes

La fonction `get_next_scrape_number` permet de déterminer le prochain numéro de scrape afin d'éviter d'écraser des fichiers existants lors de la sauvegarde.


In [ ]:
def get_next_scrape_number(output_dir, category_name):
    """Détermine le prochain numéro de scrape pour un dossier donné."""
    scrape_number = 1
    for filename in os.listdir(output_dir):
        if filename.startswith(f"{category_name}_") and filename.endswith(".csv"):
            try:
                # Extraction du numéro de scrape à partir du nom du fichier
                current_number = int(filename.split('_scrape')[-1].split('.')[0])
                if current_number >= scrape_number:
                    scrape_number = current_number + 1
            except ValueError:
                continue
    return scrape_number


## Fonction Principale de Scraping pour Flipkart

La fonction `scrape_flipkart` orchestre le scraping pour une catégorie donnée :
- Elle parcourt le nombre de pages défini.
- Pour chaque page, elle appelle `scrape_flipkart_page` afin d'extraire les produits.
- Elle accumule les résultats, puis sauvegarde les données dans un fichier CSV dans un dossier dédié à la catégorie.


In [ ]:
def scrape_flipkart(category_url, num_pages, category_name, output_dir="data/raw/flipkart"):
    aggregated_results = []
    for page in range(1, num_pages + 1):
        print(f"Scraping page {page}...")
        page_url = f"{category_url}&page={page}"
        page_results = scrape_flipkart_page(page_url, category_name)

        if not page_results:
            print("No more products found. Stopping.")
            break

        aggregated_results.extend(page_results)
        wait_random()

    if aggregated_results:
        today = datetime.today()
        formatted_date = today.strftime("%Y_%m_%d")

        # Création du dossier spécifique à la catégorie
        category_directory = os.path.join(output_dir, category_name)
        os.makedirs(category_directory, exist_ok=True)

        # Détermination du prochain numéro de scrape
        scrape_number = get_next_scrape_number(category_directory, category_name)

        filename = f"{category_name}_{formatted_date}_scrape{scrape_number}.csv"
        output_path = os.path.join(category_directory, filename)

        df = pd.DataFrame(aggregated_results)
        df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"Data saved to {output_path}")
    else:
        print("No data scraped.")

    return aggregated_results


## Script Principal

Nous définissons ici un dictionnaire `categories` qui associe à chaque catégorie son URL de base et le nombre de pages à scraper.
Ensuite, nous parcourons chaque catégorie et appelons la fonction `scrape_flipkart` pour récupérer et sauvegarder les données.


In [ ]:
if __name__ == "__main__":
    categories = {
        "graphics_cards": {
            "url": "https://www.flipkart.com/gaming-components/graphic-cards/pr?sid=4rr,tin,6zn&q=graphics+card&otracker=categorytree",
            "num_pages": 0  # Modifier le nombre de pages souhaité
        },
        "laptops": {
            "url": "https://www.flipkart.com/laptops/pr?sid=6bo,b5g&q=laptop&otracker=categorytree",
            "num_pages": 0  # Modifier le nombre de pages souhaité
        },
        "monitors": {
            "url": "https://www.flipkart.com/search?q=monitor&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off",
            "num_pages": 13
        },
        "smart_watches": {
            "url": "https://www.flipkart.com/wearable-smart-devices/smart-watches/pr?sid=ajy,buh&q=smart+watches&otracker=categorytree",
            "num_pages": 13
        }
    }

    for category_name, config in categories.items():
        print(f"Scraping {category_name}...")
        scrape_flipkart(config["url"], config["num_pages"], category_name)


## Conclusion

Ce notebook vous permet de :
- **Scraper** les données de Flipkart pour différentes catégories de produits.
- **Extraire** des informations détaillées telles que les spécifications, ratings et reviews.
- **Sauvegarder** les résultats dans des fichiers CSV organisés par catégorie.

Avant d'exécuter ce notebook, assurez-vous d'avoir installé les bibliothèques requises, par exemple en utilisant :
